# Read SpaceX Launch Data from https://api.spacexdata.com/v4

Documentation of the SpaceX API can be found @ https://github.com/r-spacex/SpaceX-API/tree/master/docs

In [70]:
import requests

# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [71]:
print(f"There are {len(launches)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

There are 205 launches
There are 225 payloads
There are 4 rockets


In [72]:
# for launch in launches:
#   print(launch.get('name'), launch.get("id"), launch.get('date_utc'), launch.get("rocket"))

FalconSat 5eb87cd9ffd86e000604b32a 2006-03-24T22:30:00.000Z 5e9d0d95eda69955f709d1eb
DemoSat 5eb87cdaffd86e000604b32b 2007-03-21T01:10:00.000Z 5e9d0d95eda69955f709d1eb
Trailblazer 5eb87cdbffd86e000604b32c 2008-08-03T03:34:00.000Z 5e9d0d95eda69955f709d1eb
RatSat 5eb87cdbffd86e000604b32d 2008-09-28T23:15:00.000Z 5e9d0d95eda69955f709d1eb
RazakSat 5eb87cdcffd86e000604b32e 2009-07-13T03:35:00.000Z 5e9d0d95eda69955f709d1eb
Falcon 9 Test Flight 5eb87cddffd86e000604b32f 2010-06-04T18:45:00.000Z 5e9d0d95eda69973a809d1ec
COTS 1 5eb87cdeffd86e000604b330 2010-12-08T15:43:00.000Z 5e9d0d95eda69973a809d1ec
COTS 2 5eb87cdfffd86e000604b331 2012-05-22T07:44:00.000Z 5e9d0d95eda69973a809d1ec
CRS-1 5eb87ce0ffd86e000604b332 2012-10-08T00:35:00.000Z 5e9d0d95eda69973a809d1ec
CRS-2 5eb87ce1ffd86e000604b333 2013-03-01T19:10:00.000Z 5e9d0d95eda69973a809d1ec
CASSIOPE 5eb87ce1ffd86e000604b334 2013-09-29T16:00:00.000Z 5e9d0d95eda69973a809d1ec
SES-8 5eb87ce2ffd86e000604b335 2013-12-03T22:41:00.000Z 5e9d0d95eda69973a

In [73]:
# for rocket in rockets:
#   print(rocket.get("name"), rocket.get("id"), rocket.get("type"), rocket.get("cost_per_launch"))

Falcon 1 5e9d0d95eda69955f709d1eb rocket 6700000
Falcon 9 5e9d0d95eda69973a809d1ec rocket 50000000
Falcon Heavy 5e9d0d95eda69974db09d1ed rocket 90000000
Starship 5e9d0d96eda699382d09d1ee rocket 7000000


In [ ]:
# for payload in payloads:
#   print(payload.get("name"), payload.get("id"), payload.get("type"), payload.get("launch"))

In [74]:
import os
import sqlite3

# Delete the old database file if it exists
if os.path.exists("spacex.db"):
    os.remove("spacex.db")
    print("Old database deleted")

Old database deleted


In [75]:
# Create a connection (this creates spacex.db in your current Colab environment)
conn = sqlite3.connect("spacex.db")

# Create a cursor
cur = conn.cursor()

schema = """
CREATE TABLE IF NOT EXISTS rockets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    cost_per_launch INTEGER
);

CREATE TABLE IF NOT EXISTS launches (
    id TEXT PRIMARY KEY,
    name TEXT,
    date_utc TEXT,
    rocket_id TEXT,
    FOREIGN KEY (rocket_id) REFERENCES rockets(id)
);

CREATE TABLE IF NOT EXISTS payloads (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    mass_kg INTEGER
);
"""

cur.executescript(schema)
conn.commit()
print("Database schema created!")

Database schema created!


In [78]:
# Insert rockets
for r in rockets:
    cur.execute("""
        INSERT OR IGNORE INTO rockets
        (id, name, type, cost_per_launch)
        VALUES (?, ?, ?, ?)
    """, (
        r["id"], r["name"], r["type"], r["cost_per_launch"]
    ))

conn.commit()
print("Rockets data inserted")

Rockets data inserted


In [79]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM rockets LIMIT 5;", conn)
df

,id,name,type,cost_per_launch
0,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,6700000
1,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,50000000
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,90000000
3,5e9d0d96eda699382d09d1ee,Starship,rocket,7000000
